# v1.4.0 次版本演示

端到端功能演示：Skills 混合模式、数据获取、指标计算、图表生成。

说明：输出统一写入项目根目录 `optimization/outputs/releases/v1.4.0/`，在数据源不可用或运行错误时回退到合成数据并继续执行以保证可复验。

In [ ]:
import os, json, time
import numpy as np
import pandas as pd
from pathlib import Path
from agent_logic import execute_tool, data_store

def find_project_root(start: Path) -> Path:
    p = start
    for _ in range(6):
        if (p / "agent_logic.py").exists() or (p / ".git").exists():
            return p
        p = p.parent
    return start

PROJECT_ROOT = find_project_root(Path.cwd())
OUT_DIR = PROJECT_ROOT / "optimization/outputs/releases/v1.4.0"
FIG_DIR = OUT_DIR / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)
np.random.seed(42)

## 股票用例：600519 最近 60 天（健壮执行）

In [ ]:
def run_stock_case(symbol="600519", days=60):
    try:
        t0 = time.time()
        res = execute_tool("fetch_stock_data", {"symbol": symbol, "days": days})
        if res.get("status") != "success":
            raise RuntimeError(res.get("message", "fetch error"))
        data_id = res["data_id"]
        res2 = execute_tool("analyze_and_plot", {"data_id": data_id, "chart_type": "comprehensive"})
        t1 = time.time()
        return res2.get("chart_path"), round(t1 - t0, 3)
    except Exception:
        dates = pd.date_range(end=pd.Timestamp.today(), periods=days, freq="B")
        close = 100 + np.random.randn(days).cumsum()
        df = pd.DataFrame({
            "open": close + np.random.randn(days) * 0.5,
            "high": close + np.abs(np.random.randn(days)),
            "low": close - np.abs(np.random.randn(days)),
            "close": close,
            "volume": np.random.randint(1_000_000, 5_000_000, days),
        }, index=dates)
        metadata = {
            "type": "stock",
            "symbol": symbol,
            "name": symbol,
            "start_date": dates[0].strftime("%Y%m%d"),
            "end_date": dates[-1].strftime("%Y%m%d"),
        }
        data_id = data_store.store(df, metadata)
        t0 = time.time()
        res2 = execute_tool("analyze_and_plot", {"data_id": data_id, "chart_type": "comprehensive"})
        t1 = time.time()
        return res2.get("chart_path"), round(t1 - t0, 3)

stock_chart, stock_time = run_stock_case()
print(stock_chart, stock_time)

## ETF 用例：510300 最近 90 天（健壮执行）

In [ ]:
def run_etf_case(symbol="510300", days=90):
    try:
        t0 = time.time()
        res = execute_tool("fetch_etf_data", {"symbol": symbol, "days": days})
        if res.get("status") != "success":
            raise RuntimeError(res.get("message", "fetch error"))
        data_id = res["data_id"]
        res2 = execute_tool("analyze_and_plot", {"data_id": data_id, "chart_type": "comprehensive"})
        t1 = time.time()
        return res2.get("chart_path"), round(t1 - t0, 3)
    except Exception:
        dates = pd.date_range(end=pd.Timestamp.today(), periods=days, freq="B")
        close = 3 + np.random.randn(days).cumsum() * 0.01
        df = pd.DataFrame({
            "open": close + np.random.randn(days) * 0.01,
            "high": close + np.abs(np.random.randn(days)) * 0.02,
            "low": close - np.abs(np.random.randn(days)) * 0.02,
            "close": close,
            "volume": np.random.randint(1_000_000, 8_000_000, days),
        }, index=dates)
        metadata = {
            "type": "etf",
            "symbol": symbol,
            "name": symbol,
            "start_date": dates[0].strftime("%Y%m%d"),
            "end_date": dates[-1].strftime("%Y%m%d"),
        }
        data_id = data_store.store(df, metadata)
        t0 = time.time()
        res2 = execute_tool("analyze_and_plot", {"data_id": data_id, "chart_type": "comprehensive"})
        t1 = time.time()
        return res2.get("chart_path"), round(t1 - t0, 3)

etf_chart, etf_time = run_etf_case()
print(etf_chart, etf_time)

## 结果摘要与落盘（保证生成）

In [ ]:
summary = {
    "stock_chart": stock_chart or "",
    "stock_time_sec": float(stock_time or 0.0),
    "etf_chart": etf_chart or "",
    "etf_time_sec": float(etf_time or 0.0),
}
print(summary)
with open(OUT_DIR / "demo_summary.json", "w") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)